In [2]:
import cv2
import numpy as np
import os
import math
from src.our_chessboard_detection import *
from matplotlib import pyplot as plt



dataDir = 'images' # Change this, according to your images' directory path

In [11]:
# Open image
image = cv2.imread(os.path.join(dataDir, "42", 'G042_IMG010.jpg')) # Change this, according to your image's path

cv2.imshow("Image", image)

cv2.waitKey(0)
cv2.destroyAllWindows()

### applying different filters

In [ ]:

# Convert to grayscale
gray_image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

# gaussian blur
# Apply Gaussian blur
# to reduce noise and improve corner detection
blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Initiate FAST object with default values
fast = cv2.FastFeatureDetector_create()

# find and draw the keypoints
kp = fast.detect(gray_image, None)
img3 = cv2.drawKeypoints(gray_image, kp, None, color=(255, 0, 0))

cv2.imshow("FAST Corner Detection w/ NMS", img3)

# Disable NMS
fast.setNonmaxSuppression(0)
kp = fast.detect(gray_image, None)
corner_detection = cv2.drawKeypoints(gray_image, kp, None, color=(255, 0, 0))

# Apply Canny filter
image_canny = cv2.Canny(image, 50, 200)


# Create RGB copy of image
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


# Display image
cv2.imshow("Original", image)
cv2.imshow("blurred", blurred)
cv2.imshow("gray_image", gray_image)
cv2.imshow("image_canny", image_canny)
cv2.imshow("rgb_image", rgb_image)
cv2.imshow("FAST Corner Detection w/o NMS", corner_detection)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Detecting chessboard

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
closed = cv2.morphologyEx(image_canny, cv2.MORPH_CLOSE, kernel)

# Find contours
contours, _ = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)

# Optional: draw all contours to debug
image_with_contours = image.copy()
cv2.drawContours(image_with_contours, contours, -1, (0, 255, 0), 3)


squares = []
area = 0
major_aprox = None
for cnt in contours:
    epsilon = 0.05 * cv2.arcLength(cnt, True)
    approx = cv2.approxPolyDP(cnt, epsilon, True)
    if len(approx) == 4 and cv2.isContourConvex(approx):
        if area < cv2.contourArea(cnt):
            major_aprox= approx
            area = cv2.contourArea(cnt)
            

print(f"Number of squares detected: {len(squares)}")
print(f"Squares: {squares}")

contours_detected = image.copy()
cv2.drawContours(contours_detected, [major_aprox], -1, (0, 0, 255), 10)
cv2.imshow("Contours Detected", contours_detected)
cv2.waitKey(0)
cv2.destroyAllWindows()
